In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

TorchVision 데이터셋을 사용

In [4]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [01:08<00:00, 384524.06it/s] 


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 104155.00it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:10<00:00, 427119.45it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



Dataset 을 DataLoader 의 인자로 전달합니다. 
이는 데이터셋을 순회 가능한 객체(iterable)로 감싸고, 자동화된 배치(batch), 샘플링(sampling), 섞기(shuffle) 및 다중 프로세스로 데이터 불러오기(multiprocess data loading)를 지원합니다. 
여기서는 배치 크기(batch size)를 64로 정의합니다. 즉, 데이터로더(dataloader) 객체의 각 요소는 64개의 특징(feature)과 정답(label)을 묶음(batch)으로 반환합니다.

In [5]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 모델 만들기

PyTorch에서 신경망 모델은 nn.Module 을 상속받는 클래스(class)를 생성하여 정의합니다.
__init__ 함수에서 신경망의 계층(layer)들을 정의하고 forward 함수에서 신경망에 데이터를 어떻게 전달할지 지정합니다. 
가능한 경우 GPU 또는 MPS로 신경망을 이동시켜 연산을 가속(accelerate)합니다.

In [6]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    # MPS는 NVIDIA GPU의 성능을 최적화하여 여러 프로세스 간에 GPU 메모리를 효율적으로 공유하는 기술
    if torch.backends.mps.is_available()
    # MPS를 확인하고, 사용 가능하다면 'mps'로 설정
    else "cpu"
    #  MPS도 사용 불가능하다면, 'cpu'로 장치를 설정
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() #  super().__init__()은 부모 클래스인 nn.Module의 초기화 메서드를 호출
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x): 
        # 모델의 순전파(forward pass)를 정의
        # 이 메서드는 입력 데이터를 받아서 모델의 출력(logits)을 계산
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
# 클래스의 인스턴스를 생성하고, to(device)를 호출하여 모델을 선택된 장치로 옮깁
# print를 확인해보았을 때 cpu에서 수행될것이다

print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


                 Input (Batch, 28, 28)
                     |
                nn.Flatten()
                     |
                 Linear(28*28, 512)
                     |
                    ReLU
                     |
                 Linear(512, 512)
                     |
                    ReLU
                     |
                 Linear(512, 10)
위 구조는 세 개의 선형 레이어로 구성된 간단한 신경망입니다. 첫 번째 선형 레이어는 28x28 크기의 이미지를 1차원으로 평탄화한 후, 512개의 뉴런으로 연결합니다. 이후 두 번째 선형 레이어와 세 번째 선형 레이어는 512개의 뉴런과 10개의 뉴런으로 연결되고, 각각 ReLU 활성화 함수를 사용합니다. 최종 출력은 크기가 10인 벡터로서, MNIST 데이터셋의 클래스(숫자 0부터 9)를 분류하는 결과를 나타냅니다.

이러한 구조는 이미지를 처리하는 간단한 신경망으로, 각 레이어와 활성화 함수를 순차적으로 연결하여 전체 모델을 정의하는 방식입니다.

## 모델 매개변수 최적화

모델을 학습하려면 손실 함수(loss function) 와 옵티마이저(optimizer) 가 필요하다

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

각 학습 단계(training loop)에서 모델은 (배치(batch)로 제공되는) 학습 데이터셋에 대한 예측을 수행하고, 예측 오류를 역전파하여 모델의 매개변수를 조정한다

In [9]:
# model = NeuralNetwork 클래스의 인스턴스

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) # 력 데이터와 레이블을 선택된 장치(CPU, GPU 또는 MPS)로 옮긴다

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad() # 옵티마이저에 저장된 모든 가중치의 기울기를 초기화
        loss.backward() # 손실 값에 대한 모델 가중치의 기울기를 계산
        optimizer.step() # 옵티마이저를 사용하여 가중치를 업데이트

        # 미니배치를 100개씩 진행할 때마다 손실 값을 출력하여 학습 진행 상황을 확인
        if batch % 100 == 0: 
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

테스트 데이터셋으로 모델의 성능 테스트

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297957  [   64/60000]
loss: 2.289633  [ 6464/60000]
loss: 2.273991  [12864/60000]
loss: 2.272156  [19264/60000]
loss: 2.242151  [25664/60000]
loss: 2.217072  [32064/60000]
loss: 2.227234  [38464/60000]
loss: 2.195457  [44864/60000]
loss: 2.193995  [51264/60000]
loss: 2.155282  [57664/60000]
Test Error: 
 Accuracy: 43.1%, Avg loss: 2.155626 

Epoch 2
-------------------------------
loss: 2.162978  [   64/60000]
loss: 2.152516  [ 6464/60000]
loss: 2.103385  [12864/60000]
loss: 2.121102  [19264/60000]
loss: 2.054420  [25664/60000]
loss: 2.000385  [32064/60000]
loss: 2.033395  [38464/60000]
loss: 1.959304  [44864/60000]
loss: 1.962134  [51264/60000]
loss: 1.881302  [57664/60000]
Test Error: 
 Accuracy: 55.8%, Avg loss: 1.891548 

Epoch 3
-------------------------------
loss: 1.920123  [   64/60000]
loss: 1.888951  [ 6464/60000]
loss: 1.785741  [12864/60000]
loss: 1.826095  [19264/60000]
loss: 1.694589  [25664/60000]
loss: 1.658640  [32064/600

## 모델 저장

모델을 저장하는 일반적인 방법은 (모델의 매개변수들을 포함하여) 내부 상태 사전(internal state dictionary)을 직렬화(serialize)하는 것 (상태 사전을 직렬화하여 바이트 스트림으로 변환)

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 모델 불러오기

모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정

In [13]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

불러온 모델을 사용한 예측

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [16]:
import torch
import torchvision.models as models

### [모델 불러오기](https://tutorials.pytorch.kr/beginner/basics/saveloadrun_tutorial.html)

PyTorch 모델은 학습한 매개변수를 state_dict라고 불리는 내부 상태 사전(internal state dictionary)에 저장합니다. 이 상태 값들은 torch.save 메소드를 사용하여 저장(persist)할 수 있습니다:

In [17]:
model = models.vgg16(weights='IMAGENET1K_V1') # ImageNet 데이터셋에서 사전 학습된 가중치
torch.save(model.state_dict(), 'model_weights.pth') # . torch.save() 함수는 이 상태 사전을 'model_weights.pth'라는 파일로 저장

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\deepBlue/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [03:00<00:00, 3.07MB/s] 


모델 가중치를 불러오기 위해서는, 먼저 동일한 모델의 인스턴스(instance)를 생성한 다음에 load_state_dict() 메소드를 사용하여 매개변수들을 불러옵니다.

In [18]:
model = models.vgg16() # 여기서는 ``weights`` 를 지정하지 않았으므로, 학습되지 않은 모델을 생성합니다.
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

추론(inference)을 하기 전에 model.eval() 메소드를 호출하여 드롭아웃(dropout)과 배치 정규화(batch normalization)를 평가 모드(evaluation mode)로 설정해야 합니다. 그렇지 않으면 일관성 없는 추론 결과가 생성됩니다.
- dropout은 과적합을 방지하기 위해 일정 뉴런의 출력을 무시하는 것이기 때문에 추론 단계에서는 사용해서는 안됨
- Batch Normalization의 경우에는 학습 단계에서는 미니배치 내에서 평균과 분산을 계산하여 뉴런의 출력을 정규화하므로 추론 단계에서는 더 이상 미니배치의 평균과 분산을 사용할 필요가 없으므로, 이러한 정규화를 제거해야 한다

## 모델의 형태를 포함하여 저장하고 불러오기

모델의 가중치를 불러올 때, 신경망의 구조를 정의하기 위해 모델 클래스를 먼저 생성(instantiate)해야 했습니다. 이 클래스의 구조를 모델과 함께 저장하고 싶으면, (model.state_dict()가 아닌) model 을 저장 함수에 전달합니다:

In [19]:
torch.save(model, 'model.pth')

모델 불러오기

In [20]:
model = torch.load('model.pth')

## reference

- https://tutorials.pytorch.kr/beginner/basics/intro.html
- https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html
- https://tutorials.pytorch.kr/beginner/basics/saveloadrun_tutorial.html